# Upload data via AnkiConnect

In [ ]:
#| default_exp anki

In [ ]:
#| export
import httpx
import json
from typing import Any
from pydantic import BaseModel

In [ ]:
#| exporti
URL = "http://localhost:8765"

def _call(
    action: str,  # AnkiConnect action name
    params: dict | None = None,  # Action parameters
    version: int = 6  # AnkiConnect API version
) -> Any:
      payload = {
          "action": action,
          "version": version,
          "params": params or {}
      }
      resp = httpx.post(URL, json=payload, timeout=10)
      resp.raise_for_status()
      data = resp.json()
      if data.get("error"):
          raise RuntimeError(f"{action} failed: {data['error']}")
      return data["result"]


def call(
    action: str,  # AnkiConnect action name
    params: dict | None = None,  # Action parameters
    version: int = 6  # AnkiConnect API version
) -> Any:
    try:
        return _call(action, params, version)
    except httpx.ConnectError:
        raise ConnectionError("\n".join([
             "Cannot connect to AnkiConnect. Please ensure:",
             "1. Anki is running",             
             "2. AnkiConnect add-on is installed",
             "3. Anki is listening on http://localhost:8765",
        ]))
    except httpx.TimeoutException:
        raise TimeoutError("AnkiConnect request timed out.")
    except RuntimeError as e:
        raise

In [ ]:
#| export
def deckNames() -> list[str]:
    """Get list of all deck names."""
    return call("deckNames")

def deleteDecks(decks: list[str]) -> None:
    """Delete specified decks and all their cards."""
    return call("deleteDecks", {"decks": decks, "cardsToo": True})

def deleteDeck(deck: str) -> None:
    """Delete specified deck and all its cards."""
    return deleteDecks([deck])

def deleteDecksAll() -> None:
    """Delete all decks and their cards."""
    return deleteDecks(call("deckNames"))

In [ ]:
#| exporti
class CardTemplate(BaseModel):
    Name: str
    Front: str
    Back: str

class CreateModelParams(BaseModel):
    modelName: str
    inOrderFields: list[str]
    css: str
    cardTemplates: list[CardTemplate]

class FinnishNoteFields(BaseModel):
    Finnish: str
    English: str
    Japanese: str
    Audio: str = ""
    Image: str = ""

class NoteOptions(BaseModel):
    allowDuplicate: bool = False

class FinnishNote(BaseModel):
    deckName: str
    modelName: str
    fields: FinnishNoteFields
    options: NoteOptions = NoteOptions()
    tags: list[str] = []

In [ ]:
#| exporti
def field(name: str, style: str = "") -> str:
    """Ankiフィールド参照を生成"""
    s = f" style='{style}'" if style else ""
    return f"<div{s}>{{{{{name}}}}}</div>"

def conditional(name: str, content: str, style: str = "") -> str:
    """条件付きコンテンツ（Anki Mustache構文）"""
    s = f" style='{style}'" if style else ""
    start = "{{#" + name + "}}"
    end = "{{/" + name + "}}"
    return f"{start}<div{s}>{content}</div>{end}"

def audio_field() -> str:
    """Audio条件付き表示"""
    return conditional("Audio", "{{Audio}}")

def image_field() -> str:
    """Image条件付き表示"""
    return conditional("Image", "<img src='{{Image}}'>", "margin-top:10px")

def finnish_large() -> str:
    """Finnish大表示"""
    return field("Finnish", "font-size:34px")

def meanings() -> str:
    """英日表示"""
    return field("English") + field("Japanese")

def full_answer() -> str:
    """完全な答え（Finnish + 英日 + 画像 + Audio）"""
    return finnish_large() + "<hr>" + meanings() + image_field() + audio_field()

In [ ]:
#| export
MODEL_NAME = "Finnish-EN-JA-Audio-Image"

In [ ]:
#| exporti
def create_finnish_model() -> CreateModelParams:
    """Finnish学習用のモデル定義を生成"""
    return CreateModelParams(
        modelName=MODEL_NAME,
        inOrderFields=["Finnish", "English", "Japanese", "Audio", "Image"],
        css=".card{font-family:arial;font-size:24px}.small{font-size:18px;color:#666} img{max-width:100%;height:auto}",
        cardTemplates=[
            # Finnish + Audio → All (一方向のみ)
            CardTemplate(
                Name="Finnish → All",
                Front=finnish_large() + audio_field(),
                Back=full_answer()
            )
        ]
    )

In [ ]:
#| export
def ensure_model(model_name: str = MODEL_NAME) -> None:
    """モデルが存在しない場合は作成"""
    if model_name not in call("modelNames"):
        params = create_finnish_model()
        call("createModel", params.model_dump())

In [ ]:
#| export
import csv
import base64
import os

def addnotes(
    deck: str,  # Anki deck name
    tsv: str    # Path to TSV file
) -> None:
    """Add notes to Anki deck from TSV file.
    
    Reads TSV file and creates Anki cards with Finnish, English, Japanese,
    audio, and image fields. Automatically creates deck if it doesn't exist.
    Uploads media files (MP3 audio and images) to Anki.
    """
    if not deck in call("deckNames"): call("createDeck", {"deck":deck})
    ensure_model()  # モデルが存在することを確認
    
    def b64(path: str) -> str:
        """Encode file to base64 string."""
        return base64.b64encode(open(path, "rb").read()).decode()
    seen = set()
    def store(path: str) -> str:
        path = (path or "").strip()
        fn = os.path.basename(path) if path else ""
        if not fn: return ""
        if path in seen: return fn
        seen.add(path)
        call("storeMediaFile",{"filename":fn,"data":b64(path)})
        return fn
    
    notes=[]
    with open(tsv, encoding="utf-8") as f:
        for r in csv.DictReader(f, delimiter="\t"):
            r = {k:(v or "").strip() for k,v in r.items()}
            fi,en,ja = r["Finnish"],r["English"],r["Japanese"]
            mp3 = store(r.get("mp3_path",""))
            img = store(r.get("img_path",""))
            
            # Parse tags from TSV (comma-separated string to list)
            tags_str = r.get("tags", "")
            tags = [t.strip() for t in tags_str.split(",")] if tags_str else []
        
            # Pydanticモデルで型安全に構築
            note = FinnishNote(
                deckName=deck,
                modelName=MODEL_NAME,
                fields=FinnishNoteFields(
                    Finnish=fi,
                    English=en,
                    Japanese=ja,
                    Audio=f"[sound:{mp3}]" if mp3 else "",
                    Image=img
                ),
                tags=tags
            )
            notes.append(note.model_dump())
    
    call("addNotes",{"notes":notes})

## EDA

In [ ]:
#| eval: false
assert(call('version')==6)

call("deleteDecks", {"decks":["NewDeck"], "cardsToo": True})
print(call('deckNames'))
call('createDeck', {"deck":"NewDeck"})
print(call('deckNames'))

['Default']
['Default', 'NewDeck']


In [ ]:
#| eval: false
call("addNote",{
    "note": {
        "deckName": "NewDeck",
        "modelName": "Basic",
        "fields": {
            "Front": "hello",
            "Back": "こんにちは",
        },
        "options": {
            "allowDuplicate": False,
        }
}})

1768677310825

In [ ]:
#| eval: false
from pydantic import BaseModel

class NoteOptions(BaseModel):
    allowDuplicate: bool = False

class NoteFields(BaseModel):
    Front: str
    Back: str

class Note(BaseModel):
    deckName: str
    modelName: str
    fields: NoteFields
    options: NoteOptions = NoteOptions()

note = Note(
    deckName="NewDeck",
    modelName="Basic",
    fields=NoteFields(Front="hello2", Back="こんにちは")
)
call("addNote", {"note": note.model_dump()})

1768677310874

In [ ]:
#| eval: false
from IPython.display import JSON

cards = call("findCards", {"query": "deck:NewDeck"})
info = call("cardsInfo", {"cards": cards})
JSON(info)

<IPython.core.display.JSON object>

In [ ]:
#| eval: false
call("deleteDecks", {"decks":["NewDeck"], "cardsToo": True})

In [ ]:
#| eval: false
deleteDecksAll()
deckNames()

['Default']

In [ ]:
#| eval: false
call("modelNames")

['Basic',
 'Basic (and reversed card)',
 'Basic (optional reversed card)',
 'Basic (type in the answer)',
 'Cloze',
 'Finnish-EN-JA-Audio-Image',
 'Image Occlusion']

In [ ]:
#| eval: false
call("modelFieldNames", {"modelName":"Basic (and reversed card)"})

['Front', 'Back']

In [ ]:
#| eval: false
call("modelFieldNames", {"modelName":"Finnish-EN-JA-Audio-Image"})

['Finnish', 'English', 'Japanese', 'Audio', 'Image']

In [ ]:
#| eval: false
call("deckNames")

['Default']

## Model（Note type

In [ ]:
#| eval: false
call("modelNames")

['Basic',
 'Basic (and reversed card)',
 'Basic (optional reversed card)',
 'Basic (type in the answer)',
 'Cloze',
 'Finnish-EN-JA-Audio-Image',
 'Image Occlusion']

AnkiConnect v6では、モデルの削除はサポートされていません。
```python
call("deleteModel", {"modelName": "Finnish-EN-JA-Audio-Image"})
```
方法: Ankiアプリから手動削除（推奨）
1. Ankiを開く
2. メニュー: Tools → Manage Note Types
3. Finnish-EN-JA-Audio-Imageを選択
4. Deleteボタンをクリック

In [ ]:
ctx = create_finnish_model().model_dump()
#JSON(ctx)
ctx["cardTemplates"][0]["Front"]

"<div style='font-size:34px'>{{Finnish}}</div>{{#Audio}}<div>{{Audio}}</div>{{/Audio}}"

In [ ]:
audio_field()

'{{#Audio}}<div>{{Audio}}</div>{{/Audio}}'

```python
ctx = {
      "modelName": "Finnish-EN-JA-Audio-Image",
      "inOrderFields": ["Finnish","English","Japanese","Audio","Image"],
      "css": ".card{font-family:arial;font-size:24px}.small{font-size:18px;color:#666} img{max-width:100%;height:auto}",
      "cardTemplates":[
        {
          "Name":"FI → EN/JA",
          "Front":"<div style='font-size:34px'>{{Finnish}}</div>{{#Audio}}<div>{{Audio}}</div>{{/Audio}}",
          "Back":"<div style='font-size:34px'>{{Finnish}}</div><hr><div>{{English}}</div><div>{{Japanese}}</div>{{#Image}}<div style='margin-top:10px'><img src='{{Image}}'></div>{{/Image}}{{#Audio}}<div style='margin-top:10px'>{{Audio}}</div>{{/Audio}}"
        },
      ]
    }
#JSON(ctx)
print(ctx["cardTemplates"][0]["Front"])
call("createModel",ctx)
```

In [ ]:
#| eval: false
# まず、Modelのフィールドを再確認（安全確認）
call("modelFieldNames", {"modelName":"Finnish-EN-JA-Audio-Image"})

['Finnish', 'English', 'Japanese', 'Audio', 'Image']

## Check TSV file content

In [ ]:
#| eval: false
tsv = "05_Lääkärissä.tsv"
tsv.split('_')

['05', 'Lääkärissä.tsv']

In [ ]:
#| eval: false
deleteDecksAll()

In [ ]:
#| eval: false
import glob
from pathlib import Path

for tsv in glob.glob('tsvs/*.tsv'):
    p1,ps = Path(tsv).stem.split('_')
    deck = f"{p1}::{ps}"
    addnotes(deck, tsv)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()